In [1]:
import collections
import heapq
import itertools
import os
import shutil
import string

import numpy as np
import pandas as pd

In [2]:
class Problem(object):
    
    def __init__(self, problem_name, has_subtests=True):
        self.problem_name = problem_name
        self.test_id = 0
        self.has_subtests = has_subtests
        if os.path.exists(self.problem_name):
            shutil.rmtree(self.problem_name)
        os.makedirs(f'{self.problem_name}/input')
        os.makedirs(f'{self.problem_name}/output')
        
    def create_inputs(self, **kwargs):
        """Should return a list. Each element is a line in the input."""
        raise NotImplementedError
    
    def solve(self, inputs, **kwargs):
        """Should return a list. Each element is a line in the output."""
        raise NotImplementedError
    
    @staticmethod
    def save_to_file(data, file_path=None, **kwargs):
        if isinstance(data, list):
            data = ' '.join(str(x) for x in data)
        print(data, file=file_path, **kwargs)
    
    def create_single_test(self, inputs=None, **kwargs):
        n = 1
        if inputs is None:
            if self.has_subtests:
                if 'N' in kwargs:
                    n = kwargs['N']
                else:
                    n = np.random.randint(kwargs['N_MIN'], kwargs['N_MAX']+1)
                inputs = [n]
                outputs = []
                for _ in range(n):
                    inp = self.create_inputs(**kwargs)
                    outp = self.solve(inp)
                    inputs += inp
                    outputs += outp
            else:
                inputs = self.create_inputs(**kwargs)
                outputs = self.solve(inputs)
        else:
            outputs = self.solve(inputs, **kwargs)
            if self.has_subtests:
                inputs = [1] + inputs

        self.write_test(self.test_id, inputs, outputs)
        self.test_id += 1
        return (self.test_id-1, inputs, outputs)
    
    def create_multiple_tests(self, n, **kwargs):
        for i in range(n):
            yield self.create_single_test(**kwargs)
            
    def write_test(self, test_id, inputs, outputs):
        input_file = f'{self.problem_name}/input/input_{test_id:02d}.txt'
        output_file = f'{self.problem_name}/output/output_{test_id:02d}.txt'

        with open(input_file, 'w') as fp:
            for line in inputs:
                Problem.save_to_file(line, fp)

        with open(output_file, 'w') as fp:
            for line in outputs:
                Problem.save_to_file(line, fp)
    
    def create_from_topcoder(self, filepath, input_parser=None, output_parser=None, max_test=50, **kwargs):
        data = pd.read_csv(filepath, sep='\t', header=None).sample(frac=1)
        for i in range(min(len(data), max_test)):
            inputs = data.iloc[i, 0] if input_parser is None else input_parser(data.iloc[i, 0])
            outputs = data.iloc[i, 1] if output_parser is None else output_parser(data.iloc[i, 1])
            self.write_test(i, inputs, outputs)
            self.test_id += 1
            yield (i, inputs, outputs)
    

def parse_topcoder_data(data):
    data = data.replace('{', '[').replace('}', ']')
    data = eval(data)
    if not isinstance(data, tuple):
        data = [data]
    return data

In [3]:
class TrainToShibuya(Problem):
    def __init__(self):
        super().__init__('train-to-shibuya', has_subtests=False)

    def create_inputs(self, n_max=10000, **kwargs):
        n = np.random.randint(n_max >> 1, n_max+1)
        return [n]
    
    def solve(self, inputs, **kwargs):
        n = inputs[0]
        MOD = int(1e9)+7

        dp = [[0] * 16 for _ in range(n+1)]
        dp[0][0] = 1
        dp[1] = [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1]
        for i in range(2, n+1):
            prev, cur = dp[i-1], dp[i]
            cur[0] = prev[15]
            cur[1] = prev[14]
            cur[2] = prev[13]
            cur[3] = (prev[15] + prev[12]) % MOD
            cur[4] = prev[11]
            cur[5] = prev[10]
            cur[6] = (prev[15] + prev[9]) % MOD
            cur[7] = (prev[14] + prev[11] + prev[8]) % MOD
            cur[8] = prev[7]
            cur[9] = prev[6]
            cur[10] = prev[5]
            cur[11] = (prev[7] + prev[4]) % MOD
            cur[12] = (prev[15] + prev[3]) % MOD
            cur[13] = (prev[14] + prev[2]) % MOD
            cur[14] = (prev[13] + prev[7] + prev[1]) % MOD
            cur[15] = (prev[15] + prev[12] + prev[6] + prev[3] + prev[0]) % MOD
        return [dp[n][15]]
            

In [4]:
problem = TrainToShibuya()

In [9]:
problem.create_single_test([3])

(18, [3], [11])

In [8]:
for _ in problem.create_multiple_tests(5, n_max=100): pass
for _ in problem.create_multiple_tests(5, n_max=1000): pass
for _ in problem.create_multiple_tests(5, n_max=10000): pass